In [2]:
%pip install -U -q "google-generativeai>=0.8.3"
!pip install chess
import google.generativeai as genai
from IPython.display import HTML, Markdown, display
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

Note: you may need to restart the kernel to use updated packages.


Checking all the available models in gemini

In [3]:
for model in genai.list_models():
    print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental
models/emb

Let's create a dictionary with 2 models so they play against each other

In [28]:
models={1:genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        temperature=0.5,top_k=8,top_p=0.8,
    )),
2: genai.GenerativeModel(
   'models/gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        temperature=0.4,top_k=8,top_p=0.9,
    ))
}


In [31]:
import chess
from google.api_core import retry

retry_policy = {
    "retry": retry.Retry(predicate=retry.if_transient_error, initial=10, multiplier=1.5, timeout=300)
}


def initialize_board():
    return chess.Board()

def get_board_state(board):
    return board.fen()

def get_move_from_model(model,board_state):
    print(board_state)
    prompt = f"""
       Given the following board state in FEN notation, suggest a legal chess move for white. 
       Use Chain of Thought and be objective, to make your move, only make valid moves.
 
        Board State (FEN):
        {board_state}
        
        Your move inside \\boxed and UCI valid with the full move including the starting and ending positions, 
        e.g., 'e2e4' or 'g1f3' for a knight move.
        """
    response = model.generate_content(prompt, request_options=retry_policy)
    print(response.text)

    if "\\boxed{" in response.text:
        move = response.text.split("\\boxed{")[1].split("}")[0].strip()
    
    print("Move suggested:", move)
    return move


def play_chess():
    board = initialize_board()
    turn = 0
    
    while not board.is_game_over():
        current_player = f"Model {turn + 1}"
        board_state = get_board_state(board)
        move = get_move_from_model(models[turn+1],board_state)
        if chess.Move.from_uci(move) in board.legal_moves:
            board.push(chess.Move.from_uci(move))
            print(f"{current_player} played: {move}")
            print(board)
        else:
            print(f"{current_player} attempted an illegal move: {move}")
        
        turn = 1 - turn

play_chess()


rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Chain of Thought:

1. **White has the first move.**  This means they can move any of their pieces.
2. **The board is completely empty except for the starting pieces.** This means there are no immediate threats or captures to consider.
3. **A common opening move for White is to move their King's pawn two squares forward.** This opens up the center of the board and allows for further development.

Therefore, the best move for White is: 
 
  \boxed{e2e4} 

Move suggested: e2e4
Model 1 played: e2e4
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1
Chain of Thought:

1. **Analyze the board:** Black has just moved, and it's White's turn. White has all their pieces on the starting rank, while Black also has all their pieces on the starting rank. 
2. **Identify possible moves:** White can move any of th

InvalidMoveError: expected uci string to be of length 4 or 5: 'Nf3'